In [ ]:
import math
import pandas as pd
import numpy as np
import pgljupyter
import openalea.plantgl.all as pgl
from vmapplet.tools.surface import leafSurface

device manual: https://polhemus.com/_assets/img/FASTRAK_User_Manual_OPM00PI002-G.pdf

In [ ]:
branches = pd.read_csv('branches.csv')
leaves = pd.read_csv('leaves.csv')

In [ ]:
branches

In [ ]:
#print(branches['code rameau'].unique())

In [ ]:
leaves

In [ ]:
# az,el are expected in degrees, in the North-clocwise convention
# In the scene, positive rotations are counter-clockwise
# north is the angle (degrees, positive counter_clockwise) between X+ and North
from math import radians

def azimuth(az, north = 0):
    return radians(az)

def elevation(el): # elevation is in counter-clockwize ????
    return radians(el)

def roll(rl):
    return radians(rl)

def position(x,y,z): # reference is inverted. All of it. However, it is maybe now oriented toward the south. we do not care for relative orientation
    return -x,-y,-z

In [ ]:
def get_leaf_geom(s=1):
    import math
    w = s/4
    l = s
    points = [(0,-w/10,0), (0,w/10,0), (l/2,w/2,0), (l/2,-w/2,0), (l,w/2,0), (l,-w/2,0)]
    indices = [(0, 1, 2, 3),(3, 2, 4, 5)]
    colors = [(0, 59, 6,0),(60, 0, 0,0)] 
    geom = pgl.QuadSet(points,indices,colorList=colors)
    return geom

leaf_smb = get_leaf_geom()

In [ ]:
branch_shapes = []
for i, branch in branches.iterrows():
    if not i % 2:
        x0 = branch['X']
        y0 = branch['Y']
        z0 = branch['Z']
    else:
        x1 = branch['X']
        y1 = branch['Y']
        z1 = branch['Z']
        geom = pgl.Extrusion(pgl.Polyline([position(x0, y0, z0), position(x1, y1, z1)]), pgl.Polyline2D.Circle(1))
        shape = pgl.Shape(geom, pgl.Material(ambient=(49,29,25) , diffuse = 2.55102, shininess=0))
        branch_shapes.append(shape)

In [ ]:
leaf_shapes = []
for _, leaf in leaves.iterrows():
    x = -leaf['X']
    y = -leaf['Y']
    z = -leaf['Z']
    a = azimuth(leaf['A'])
    b = elevation(leaf['B'])
    c = roll(leaf['C'])
    geom = pgl.Translated(x, y, z, pgl.EulerRotated(a, b, c, pgl.Scaled(5,leaf_smb)))
    shape = pgl.Shape(geom, pgl.Material(ambient=(0, 59, 6), shininess=0, diffuse=1.59322))
    leaf_shapes.append(shape)

In [ ]:
pgljupyter.SceneWidget(branch_shapes + leaf_shapes, scale=1, size_world=-branches['Z'].min())

In [ ]:
from math import radians, degrees, cos, sin

from openalea.plantgl.all import Vector3, cross
def directionFromAngles(teta, phi, psi):
    direction = (cos(teta)*cos(phi),sin(teta)*cos(phi), -sin(phi))

    direction_s = (-sin(teta)*cos(psi)+cos(teta)*sin(phi)*sin(psi),
               cos(teta)*cos(psi)+sin(teta)*sin(phi)*sin(psi),
               sin(psi)*cos(phi))
    return direction, direction_s, cross(direction, direction_s)

def plot_single_branch(code: str, widget: pgljupyter.SceneWidget, colormap = None):
    """branch bein and leaves translated to orgin 0,0,0"""
    branch = branches[branches['code rameau'] == code]
    leaves_ = leaves[leaves['code rameau'] == code]
    shapes = []

    x0 = branch.iloc[0]['X']
    y0 = branch.iloc[0]['Y']
    z0 = branch.iloc[0]['Z']
    x1 = (branch.iloc[1]['X'] - x0)  # remove offset to plot branch at origin (0,0,0)
    y1 = (branch.iloc[1]['Y'] - y0)
    z1 = (branch.iloc[1]['Z'] - z0)
    
    geom = pgl.Extrusion(pgl.Polyline([(0, 0, 0), position(x1, y1, z1)]), pgl.Polyline2D.Circle(1))
    shape = pgl.Shape(geom, pgl.Material(ambient=(49,29,25) , diffuse = 2.55102, shininess=0))
    shapes.append(shape)
    
    leafsmb = get_leaf_geom(4)
    for i, leaf in leaves_.iterrows():
        x = (leaf['X'] - x0)
        y = (leaf['Y'] - y0)
        z = (leaf['Z'] - z0)
        a = azimuth(leaf['A'])
        b = elevation(leaf['B'])
        c = roll(leaf['C'])
        dp, ds, dt = directionFromAngles(a,b,c)
        #  leafSurface()
        geom = pgl.Translated(position(x, y, z), pgl.EulerRotated(a, b, c, leafsmb))
        #geom = pgl.Translated(position(x, y, z), pgl.Oriented(ds, dt, leafsmb))
        if colormap is None:
            shape = pgl.Shape(geom, pgl.Material(ambient=(0, 59, 6), shininess=0, diffuse=1.59322))
        else:
            shape = pgl.Shape(geom, pgl.Material(ambient=colormap(i), shininess=0, diffuse=1.59322))            
        shapes.append(shape)

    widget.set_scenes(pgl.Scene(shapes), scales=0.1)

def plot_tree(code: str, widget: pgljupyter.SceneWidget, colormap = None):
    """branch bein and leaves translated to orgin 0,0,0"""
    branch = branches[branches['code rameau'] == code]
    leaves_ = leaves[leaves['code rameau'] == code]
    shapes = []

    x0 = branch.iloc[0]['X']
    y0 = branch.iloc[0]['Y']
    z0 = branch.iloc[0]['Z']
    x1 = (branch.iloc[1]['X'] - x0)  # remove offset to plot branch at origin (0,0,0)
    y1 = (branch.iloc[1]['Y'] - y0)
    z1 = (branch.iloc[1]['Z'] - z0)
    
    geom = pgl.Extrusion(pgl.Polyline([(0, 0, 0), position(x1, y1, z1)]), pgl.Polyline2D.Circle(1))
    shape = pgl.Shape(geom, pgl.Material(ambient=(49,29,25) , diffuse = 2.55102, shininess=0))
    shapes.append(shape)
    
    leafsmb = get_leaf_geom(4)
    for i, leaf in leaves_.iterrows():
        x = (leaf['X'] - x0)
        y = (leaf['Y'] - y0)
        z = (leaf['Z'] - z0)
        a = azimuth(leaf['A'])
        b = elevation(leaf['B'])
        c = roll(leaf['C'])
        dp, ds, dt = directionFromAngles(a,b,c)
        #  leafSurface()
        geom = pgl.Translated(position(x, y, z), pgl.EulerRotated(a, b, c, leafsmb))
        #geom = pgl.Translated(position(x, y, z), pgl.Oriented(ds, dt, leafsmb))
        if colormap is None:
            shape = pgl.Shape(geom, pgl.Material(ambient=(0, 59, 6), shininess=0, diffuse=1.59322))
        else:
            shape = pgl.Shape(geom, pgl.Material(ambient=colormap(i), shininess=0, diffuse=1.59322))            
        shapes.append(shape)

    widget.set_scenes(pgl.Scene(shapes), scales=0.1)

In [ ]:
single_branch_widget = pgljupyter.SceneWidget()
single_branch_widget

In [ ]:
plot_single_branch('F36PL2', single_branch_widget)

In [ ]:
plot_single_branch('F42PL16', single_branch_widget)

In [ ]:
leaf_widget = pgljupyter.SceneWidget()
leaf_widget

In [ ]:
leaf = pgl.Shape(get_leaf_geom(), pgl.Material(ambient=(0, 59, 6), shininess=0, diffuse=1.59322))
leaf_widget.set_scenes(leaf, scales=0.1)

In [ ]:
from math import radians, degrees
leaf_angles = {}
for i, leaf in leaves.iterrows():
    x = -leaf['X']
    y = -leaf['Y']
    z = -leaf['Z']
    
    a = azimuth(leaf['A'])
    e = elevation(leaf['B'])
    r = roll(leaf['C'])
    
    branch = branches[branches['code rameau'] == leaf['code rameau']]
    x0 = -branch.iloc[0]['X']
    y0 = -branch.iloc[0]['Y']
    z0 = -branch.iloc[0]['Z']
    
    x1 = (-branch.iloc[1]['X'] - x0)  # remove offset to plot branch at origin (0,0,0)
    y1 = (-branch.iloc[1]['Y'] - y0)
    z1 = (-branch.iloc[1]['Z'] - z0)
    
    #direction = (cos(phi)*cos(teta),cos(phi)*sin(teta), -sin(phi))

    xl = np.cos(e)*np.cos(a)
    yl = np.cos(e)*np.sin(a)
    zl = -np.sin(e)
    
    leaf_angles[i] = degrees(pgl.angle((x1,y1,z1),(xl,yl,zl)))

In [ ]:
from matplotlib.pyplot import *
from numpy import mean
_, _, fig = hist(leaf_angles.values(), range(0,181,10))
show()
fig2  =boxplot(leaf_angles.values())
show()

In [ ]:
def leafcolor(i):
    if leaf_angles[i] < 90:
        return (0, 59, 6)
    else:
        return (59, 0, 6)
        

from openalea.plantgl.all import PglColorMap, Color3
mymap = PglColorMap(0,18, 'jet')
def leafcolor(i):
    res = mymap(int(leaf_angles[i]//10))
    return Color3(res.red,res.green,res.blue)
        


In [ ]:
sw = pgljupyter.SceneWidget()
sw

In [ ]:
code_rameau = 'F48VL21'
plot_single_branch(code_rameau, sw, colormap = leafcolor)

In [ ]:
hist([leaf_angles[i] for i, leaf in leaves[leaves['code rameau']==code_rameau].iterrows()], range(0,181,10))